# Segmenting and Clustering Neighborhoods in Toronto

## Applied Data Science Capstone - IBM (Coursera)

### Week 3 Assignment

# Part 1: Web Scraping and Data Cleansing 

In [214]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Scraping the Wikipedia page

### Transform data into panda dataframe

### Reading data from html, making first row as a header and converting `Not assigned` to null values(NaN)


In [227]:
# Reading data from html, first row as header and changing 'Not assigned' to NaN values 
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0, na_values='Not assigned')
df = data[0]  
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Changing columns name

In [228]:
df.rename(columns= {'Postcode':'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing rows that have null value on `Borough` column

In [232]:
# Dropping NaN values from the column 'Borough'
df.dropna(axis=0, subset=['Borough'], inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### M7A row has null value on Neighborhood column. We will replace it with the value of the corresponding Borough column

In [235]:
# fill NaN value in "Neighborhood" column with the value of "Borough" column 
df.Neighborhood.fillna(df.Borough, inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Grouping the dataframe by `PostalCode` and `Borough`
### Some of the neighborhoods exist in same postalcode area. For example, M5A is listed twice and has two neighborhoods: Harbourfront and Regen Park.
#### Those neighborhoods are concatenate into same row using apply() seperated by comma.


In [238]:
# Grouping dataframe by "Postcode" and then "Borough", applying apply() and resetting index
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head(10)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


## Finding total number of rows on cleaned Dataframe

In [239]:
df.shape   # checking the rows of dataframe

(103, 3)

In [240]:
print("The cleaned dataframe has {} rows.".format(df.shape[0]))

The cleaned dataframe has 103 rows.


# Part 2: Getting the latitude and the longitude coordinates of each                                       neighborhood

## Reading csv file containing the geopastial coordinates of each postal code

In [241]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [242]:
# Renaming column "Postal Code"
coordinates.rename(columns= {'Postal Code':'PostalCode'}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [243]:
coordinates.shape  # checking the rows of the dataframe

(103, 3)

# Using merge to join columns of these two dataframes

In [244]:
df_coordinate = pd.merge(df, coordinates, how = 'inner', on="PostalCode")
df_coordinate.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#  Part 3: Explore and Cluster the neighborhoods in Toronto

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [245]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of New York with neighborhoods superimposed on top.

In [246]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coordinate['Latitude'], df_coordinate['Longitude'], df_coordinate['Borough'], df_coordinate['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

#### Define Foursquare Credentials and Version

In [249]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'
radius = 500
LIMIT = 100

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: '+ CLIENT_SECRET)

Your credentials:
CLIENT_ID: J1245ZE1HFQNEHH1D30MDNBXGBWY43ZWTEHOMJHBGVNIMREA
CLIENT_SECRET: PQFA4ZT0SN3AMC53KBZPCV4BWO1D4320BIXWOJ3UZOV2JZCU


In [271]:
venues_list=[]
for lat, lng,postal, borough, neighborhood in zip(df_coordinate['Latitude'], df_coordinate['Longitude'], 
                                                  df_coordinate['PostalCode'],df_coordinate['Borough'], 
                                                  df_coordinate['Neighborhood']):
    
   
            
     # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    for v in results:
        venues_list.append((
        postal,
        borough,
        neighborhood,
        lat, 
        lng,            
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']))



In [275]:
vdf=pd.DataFrame(venues_list)
vdf.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 
             'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
vdf.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


Let's check how many venues were returned for each postalcode

In [276]:
vdf.groupby('PostalCode').count()

,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,,,,,,,,
M1B,1,1,1,1,1,1,1,1
M1C,1,1,1,1,1,1,1,1
M1E,8,8,8,8,8,8,8,8
M1G,4,4,4,4,4,4,4,4
M1H,7,7,7,7,7,7,7,7
M1J,2,2,2,2,2,2,2,2
M1K,4,4,4,4,4,4,4,4
M1L,9,9,9,9,9,9,9,9
M1M,2,2,2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues

In [277]:
# unique catrgories
print('There are {} unique categories.'.format(len(vdf['VenueCategory'].unique())))

There are 274 unique categories.


## 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.